| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png "View source on GitHub")](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate_all.ipynb) | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png "Notebook Viewer")](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate.ipynb) | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png "Run in binder")](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate.ipynb) | [![Colab logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/colab_logo.png "Run in Google Colab")](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate.ipynb) |
|:---------------------:|:---------------:|:-------------:|:-------------------:|
| [View source on GitHub](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate_all.ipynb) | [Notebook Viewer](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate_all.ipynb) | [Run in binder](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate_all.ipynb) | [Run in Google Colab](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate_all.ipynb) |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

It plots the last-known location of all your devices on a map, using both GPS information and cell tower information, if available.

It uses [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) to display the interactive map.

It uses Google's [Geolocation API](https://developers.google.com/maps/documentation/geolocation/intro) to determine location from cell tower information.

### Requirements
* You need to have an existing user account on the WA/DMP instance.
* In order to see something on the map, you need one or more devices with GPS *and/or* a cellular connection.
* In order to plot location based on cell tower information, you must have an API key for Google's Geolocation API. (The Geolocation API is free if you make less than 40,000 calls per month).

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Then run the cells, either one at a time, or all at once.

When prompted, enter the required User Input (USERNAME, PASSWORD, GOOGLE_KEY).

The map will be displayed when you run the "Display Map" cell.

The map is dynamically updated as devices are added.

"Upside-down" markers are used to display GPS location.
"Normal" markers are used to display cell tower location.

Cell tower location is also marked with a bounding circle. i.e. The device could be anywhere inside this circle.

Hover over a marker to see the MAC addres of the device.

## Setup

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install ipyleaflet

import requests
import json
from ipyleaflet import Map, Marker, MarkerCluster, Circle

## Functions to be re-used later

In [ ]:
def login(username, password):
    """Login to the system, and return a token
    """
    url = f"{BASE_URL}/public/auth/connect/token"
    credentials = {'username': username, 'password': password, 'client_id': 'python', 'grant_type': 'password'}
    print(f"\nSending POST request to {url} with:\n"
          f"    credentials={credentials}")
    response = requests.post(url, data=credentials)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()["access_token"]
    else:
        print("Failed to login!")
        sys.exit(1)


def get_devices(page_size):
    """Retrieves the list of your devices.
       Requests are paged, but this function automatically aggregates responses into one complete list.
    """
    page_number = 1
    total, devices = get_one_page_of_devices(page_number, page_size)
    
    while len(devices) < total:
        print(f"{len(devices)} out of {total} ...")
        page_number += 1
        total, page = get_one_page_of_devices(page_number, page_size)
        devices.extend(page)

    return devices


def get_one_page_of_devices(page_number, page_size):
    """Retrieves one page of the list of your devices.
    """
    url = f"{BASE_URL}/{BASE_PATH}/management/devices"
    header = {'Authorization': f'Bearer {USER_TOKEN}'}

    # The only REQUIRED query parameters are page and pageSize
    print(f"Sending GET request to {url} with:\n"
          f"    header={header}\n"
          f"    page={page_number}\n"
          f"    pageSize={page_size}")
    query = {'page': page_number, 'pageSize': page_size}
    response = requests.get(url, params=query, headers=header)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)
    
    total = response.json()['total_items']

    if response.status_code == requests.codes['ok']:
        return total, response.json()['data']
    else:
        print(f"Failed to retrieve page {page_number}!")
        return None

    
def get_monitoring_data(influx_query):
    """Queries for monitoring data from a specific device
    """
    url = f"{BASE_URL}/{BASE_PATH}/monitoring/devices/query"
    header = {'Authorization': f'Bearer {USER_TOKEN}'}
    query = {'Q': influx_query, 'Epoch': 'ms'}
    print(f"\nSending GET request to {url} with:\n"
            f"    header={header}\n"
            f"    Q={influx_query}\n"
            f"    Epoch=ms\n")
    response = requests.get(url, params=query, headers=header)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        if response.json()['results'][0]['series']:
            return response.json()['results'][0]['series'][0]
        else:
            print("No data")
            return None
    else:
        print("InfluxDB query failed!")
        return None

def lookup_geolocation(body, key):
    """Uses Google's Geolocation API to find a Latitude and Longitude based on cell tower information
    """
    url = "https://www.googleapis.com/geolocation/v1/geolocate"
    query = {'key': key}
    print(f"\nSending POST request to {url} with:\n"
            f"    body={body}")
    response = requests.post(url, params=query, json=body)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()
    else:
        print("Google API query failed!")
        return None

## Global variables

In [ ]:
BASE_URL = 'https://gateway.staging.wadmp.com'
BASE_PATH = 'api'

## User input

In [ ]:
USERNAME = input("Enter WA/DMP username:")
PASSWORD = input("Enter WA/DMP password:")
GOOGLE_KEY = input("Enter your Google API key:")

## Display map

In [ ]:
m = Map(center=(0, 0), zoom=1)
m

## Get list of devices

In [ ]:
# Login
USER_TOKEN = login(USERNAME, PASSWORD)

# Get list of devices
my_devices = get_devices(100)

total_devices = len(my_devices)
print(f"Total number of devices = {total_devices}")

## Iterate over list, looking for GPS information

In [ ]:
devices_with_gps = 0
gps_markers = ()  # Empty tuple

for device in my_devices:
    
    # Get the latest location data from InfluxDB
    """Note that the company name must be included in the WHERE clause!
    (If you do not specify a company, the query will default to use the InfluxDB database for your primary company).

    And because of the way Grafana [uses variables in queries](https://grafana.com/docs/grafana/latest/features/datasources/influxdb/#using-variables-in-queries), you have to wrap the company name as follows:
    `"companyName" =~ /^My Company Inc.$/`

    Following the Influx recommendations [here](https://docs.influxdata.com/influxdb/v1.7/troubleshooting/frequently-asked-questions/#when-should-i-single-quote-and-when-should-i-double-quote-in-queries), we single-quote string values and double-quote identifiers.
    """
    mac = device['mac_address']
    if device['company']:
        company = device['company']['name']
    else:
        continue  # Stop curent iteration of for loop
    
    influx_query = f'SELECT LAST("gpsLatitude"), LAST("gpsLongitude") FROM "SNMP" WHERE ("macAddress" = \'{mac}\' AND "companyName" =~ /^{company}$/)'
    influx_response = get_monitoring_data(influx_query)
    
    if not influx_response:
        continue  # Stop curent iteration of for loop
        
    [timestamp, lat, lng] = influx_response['values'][0]
    print(
        f"gpsLatitude = {lat}, " \
        f"gpsLatitude = {lng}"
    )
    devices_with_gps += 1
    
    # Update the map.
    marker = Marker(location=(lat, lng), title=mac, rise_on_hover=True, draggable=False, rotation_angle=180.0, rotation_origin='bottom center')
    gps_markers += (marker,)
    
marker_cluster = MarkerCluster(markers=gps_markers)
m.add_layer(marker_cluster)


## GPS Summary

In [ ]:
print(f"{devices_with_gps} devices out of {total_devices} have GPS information")

## Iterate over list, looking for cellular information

In [ ]:
devices_with_cellid = 0
cell_markers = ()  # Empty tuple

for device in my_devices:
    
    # Get the latest location data from InfluxDB
    """Note that the company name must be included in the WHERE clause!
    (If you do not specify a company, the query will default to use the InfluxDB database for your primary company).

    And because of the way Grafana [uses variables in queries](https://grafana.com/docs/grafana/latest/features/datasources/influxdb/#using-variables-in-queries), you have to wrap the company name as follows:
    `"companyName" =~ /^My Company Inc.$/`

    Following the Influx recommendations [here](https://docs.influxdata.com/influxdb/v1.7/troubleshooting/frequently-asked-questions/#when-should-i-single-quote-and-when-should-i-double-quote-in-queries), we single-quote string values and double-quote identifiers.
    """
    mac = device['mac_address']
    if device['company']:
        company = device['company']['name']
    else:
        continue  # Stop curent iteration of for loop
    
    influx_query = f'SELECT LAST("mobileTechnology"), LAST("mobilePLMN"), LAST("mobileLAC"), LAST("mobileCell") FROM "SNMP" WHERE ("macAddress" = \'{mac}\' AND "companyName" =~ /^{company}$/)'
    influx_response = get_monitoring_data(influx_query)
    
    if not influx_response:
        continue  # Stop curent iteration of for loop
    
    [timestamp, mobileTechnology, PLMN, locationAreaCode, cellId] = influx_response['values'][0]
    mobileCountryCode = PLMN[:3]
    mobileNetworkCode = PLMN[3:]
    print(
        f"mobileTechnology = {mobileTechnology}, " \
        f"mobileCountryCode = {mobileCountryCode}, " \
        f"mobileNetworkCode = {mobileNetworkCode}, " \
        f"locationAreaCode = {locationAreaCode}, " \
        f"cellId = {cellId}"
    )

    if not(cellId and locationAreaCode):
        continue  # Stop curent iteration of for loop
    
    devices_with_cellid += 1
    
    # Use Google's Geolocation API
    """See https://developers.google.com/maps/documentation/geolocation/intro
    Note that when using requests.post with a json argument, the Content-Type header will be set to application/json automatically.
    """
    body = {
      "cellTowers": [
        {
            "cellId": int(cellId, 16),
            "locationAreaCode": int(locationAreaCode, 16),
            "mobileCountryCode": mobileCountryCode,
            "mobileNetworkCode": mobileNetworkCode,
        }
      ]
    }
    google_response = lookup_geolocation(body, GOOGLE_KEY)
    if not google_response:
        continue
    lat = google_response['location']['lat']
    lng = google_response['location']['lng']
    accuracy = google_response['accuracy']
    
    # Update the map.
    # The accuracy, in meters, represents the radius of a circle around the given location.
    circle = Circle()
    circle.location = (lat, lng)
    circle.radius = accuracy
    m.add_layer(circle)
    marker = Marker(location=(lat, lng), title=mac, rise_on_hover=True, draggable=False)
    cell_markers += (marker,)
    
marker_cluster = MarkerCluster(markers=cell_markers)
m.add_layer(marker_cluster)

## Cellular Summary

In [ ]:
print(f"{devices_with_cellid} devices out of {total_devices} have cell tower information")